In [27]:
import pickle
import keras
from keras.models import load_model
import pyaudio
import wave
import librosa
import keras.backend as K
import numpy as np
import time

sr                = 22050
n_fft             = 2048
hop_length        = 512
n_mels            = 96 # model.layers[0].input_shape[1]
slice_size        = 216 # model.layers[0].input_shape[2]
slice_time        = slice_size * hop_length / sr
epsilon_distance  = 0.001
sample_start      = 10 # 5 second samples
sample_end        = 16 # 5 second samples
sample_step       = 1 # 5 second samples
record_seconds    = 30

mp3tovec = pickle.load(open('..\Pickles\mp3tovecs\mp3tovec.p', 'rb'))
model = load_model('../speccy_model')
import os
dump_directory = '../Pickles'
mp3s = {}
for filename in os.listdir(dump_directory):
    if not os.path.isfile(dump_directory + '/' + filename):
        continue
    p = pickle.load(open(dump_directory + '/' + filename, 'rb'))
    mp3s[p[0]] = p[1]
print('Recording...')
while True:
    audio = pyaudio.PyAudio()
    stream = audio.open(format=pyaudio.paInt16, channels=1, rate=sr, input=True)
    data = stream.read(sr * record_seconds)
    start = time.time()
    stream.stop_stream()
    stream.close()
    audio.terminate()
    waveFile = wave.open('temp.wav', 'wb')
    waveFile.setnchannels(1)
    waveFile.setsampwidth(audio.get_sample_size(pyaudio.paInt16))
    waveFile.setframerate(sr)
    waveFile.writeframes(data)
    waveFile.close()
    y, sr = librosa.load('temp.wav', mono=True)
#    os.remove('temp.wav')
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels, fmax=sr/2)
    x = np.ndarray(shape=(S.shape[1] // slice_size, n_mels, slice_size, 1), dtype=float)
    for slice in range(S.shape[1] // slice_size):
        log_S = librosa.power_to_db(S[:, slice * slice_size : (slice+1) * slice_size], ref=np.max)
        if np.max(log_S) - np.min(log_S) != 0:
            log_S = (log_S - np.min(log_S)) / (np.max(log_S) - np.min(log_S))
        x[slice, :, :, 0] = log_S
    new_vecs = model.predict(x)
    new_idfs = []
    for vec_i in new_vecs:
        idf = 1 # because each new vector is in the new mp3 by definition
        for mp3 in mp3s:
            for vec_j in mp3s[mp3][sample_start:sample_end:sample_step]:
                if 1 - np.dot(vec_i, vec_j) / (np.linalg.norm(vec_i) * np.linalg.norm(vec_j)) < epsilon_distance:
                    idf += 1
                    break
        new_idfs.append(-np.log(idf / (len(mp3s) + 1))) # N + 1
    vec = 0
    for i, vec_i in enumerate(new_vecs):
        tf = 0
        for vec_j in new_vecs:
            if 1 - np.dot(vec_i, vec_j) / (np.linalg.norm(vec_i) * np.linalg.norm(vec_j)) < epsilon_distance:
                tf += 1
        vec += vec_i * tf * new_idfs[i]
    similar = most_similar_by_vec([vec], topn=1, noise=0)
    print(f'{similar[0][0]}, [{similar[0][1]:.2f}] {time.time() - start:.2f}s')

Recording...
H:\Music\Compilations\Jazz & Milk Breaks Vol 3 Sampler\01 You Can Dance (Envee Ensemble Ver.mp3, [0.99] 4.33s


KeyboardInterrupt: 